In [1]:
import sys
sys.path.append("..")
import pfb
import numpy

In [3]:
def A(x):
    """Applies PFB, irfft's that, flatten."""
    # Forward PFB the Signal
    b = pfb.forward_pfb(x)
    # Inverse Fourier Transform along axis=1
    b = irfft(b)
    # Apply circulant boundary conditions
    b = np.concatenate([b, b[:3, :]], axis=0)
    return b.flatten()
    
def A_inv(b_flat, lblock=2048):
    """Inverse of A. Reshape the array, rfft, iPFB."""
    # Sanity check
    if len(b_flat)/lblock != len(b_flat)//lblock: 
        raise Exception("Dimensions of input do not match lblock!")
    # Reshape array so that it looks like irfft'd pfb output dims
    b = b_flat.reshape((-1,lblock))[:-3,:]
    # Rfft along axis=1
    b = rfft(b)
    return pfb.inverse_pfb(b)

def A_inv_weiner(b_flat, lblock=2048):
    """Inverse of A with weiner filtering. Reshape the array, rfft, iPFB with weiner filter."""
    # Sanity check
    if len(b_flat)/lblock != len(b_flat)//lblock: 
        raise Exception("Dimensions of input do not match lblock!")
    # Reshape array so that it looks like irfft'd pfb output dims
    b = b_flat.reshape((-1,lblock))[:-3,:]
    # Rfft along axis=1
    b = rfft(b)
    return pfb.inverse_pfb(b, weiner_thresh=0.25)
    
def A_quantize(x, delta):
    """Takes signal, pfb's it, quantizes, irfft's that."""
    # Forward PFB the signal
    b = pfb.forward_pfb(x)
    # Quantize the filter bank
    # The sqrt is to account for the next IRFFT step
    b = pfb.quantize(b, np.sqrt(2*(b.shape[1] - 1)) * delta)
    # Inverse Fourier Transform
    b = irfft(b) # Same as apply along axis=1
    # Apply circulant boundary conditions
    b = np.concatenate([b, b[:3, :]], axis=0)
    return b.flatten() 

def R(x, lblock):
    """Rotation matrix?"""
    lx = len(x)
    if lx/lblock != lx//lblock: 
        raise Exception("Len x must divide lblock.")
    k = lx // lblock
    """TBC here..."""

In [ ]:
def simulate_quantization_error( delta=0.5 , k=80 ):
    """Simulates and plots the iPFB quantization noise and the noise after correction. 
    
    Parameters
    ----------
    delta : float
        The length of the interval over which quantization noise is uniformly sampled. 
        The standard deviation of this distribution is delta / root 12. 
    k : int
        The number of blocks to simulate. 
    
    """
    # Simulated input data is randomly sampled from a normal distribution. 
    x = np.random.normal(0,1,LBLOCK*k) 
    
    d = A_quantize(x,delta) 
    # N_inv and Q_inv are diagonal matrices so we store them as 1D-arrays 
    N_inv = np.ones(len(x)) * 6 / delta**2 

    ### 3 percent of original data given as prior 
    # the noise matrix for the prior 
    prior_3 = np.zeros(len(x)) # what we know about x, information we saved 
    prior_3[saved_idxs_3] = pfb.quantize_real(x[saved_idxs_3].copy() , delta) # quantized original signal 

    Q_inv_3 = np.ones(len(x)) # this is a prior, change to zeros if you want zero for infinite uncertainty
#         Q_inv_3 = np.zeros(len(x))
    Q_inv_3[saved_idxs_3] = np.ones(len(saved_idxs_3)) * (12 / delta**2) # 8 bits per real number (finer std because no complex) 

    B_3 = lambda ts: AT(N_inv * A(ts)) + Q_inv_3 * ts # think ts===x
    u_3 = AT(N_inv * d) + Q_inv_3 * prior_3 # this is same as mult prior by var=12/delta^2

    ### 1 percent of original data given as prior
    # the noise matrix for the prior
    prior_1 = np.zeros(len(x)) # what we know about x, information we saved
    prior_1[saved_idxs_1] = pfb.quantize_real(x[saved_idxs_1].copy() , delta) # quantized original signal

    Q_inv_1 = np.zeros(len(x)) 
    Q_inv_1[saved_idxs_1] = np.ones(len(saved_idxs_1)) * 12 / delta**2 # 8 bits per real number

    B_1 = lambda ts: AT(N_inv * A(ts)) + Q_inv_1 * ts # think ts===x
    u_1 = AT(N_inv * d) + Q_inv_1 * prior_1

    ### Optimize CHI squared using conjugate gradient method
    # x0 is the standard IPFB reconstruction
    x0 = np.real( A_inv(d) )
    x0_weiner = np.real( A_inv_weiner(d) ) # by default the weiner threshold is set to 0.25

    # print("\n\nd={}".format(d)) # trace, they are indeed real
    # print("\n\nx_0={}".format(x0)) # complex dtype but zero imag componant

#         print("\nConjugate Gradient Descent, with 3% extra data (prior is a quantized 3% of original timestream)")
    if verbose: 
        plt.figure(figsize=(14,3.5))

        # rms virgin pfb
        rms_virgin = (x - x0)**2
        rms_virgin = np.reshape(rms_virgin[5*LBLOCK:-5*LBLOCK],(k-10,LBLOCK)) # bad practice to hard code k=80...? I just want to write this fast
        rms_net_virgin = np.sqrt(np.mean(rms_virgin))
        rms_virgin = np.sqrt(np.mean(rms_virgin,axis=0))
        rms_virgin = mav(rms_virgin,5)
        plt.semilogy(rms_virgin[5:-5],label="rmse virgin ipfb") 

        # rms weiner filtered pfb
        rms_weiner = (x - x0_weiner)**2
        rms_weiner = np.reshape(rms_weiner[5*LBLOCK:-5*LBLOCK],(k-10,LBLOCK)) 
        rms_net_weiner = np.sqrt(np.mean(rms_weiner))
        rms_weiner = np.sqrt(np.mean(rms_weiner,axis=0))
        plt.semilogy(rms_weiner[5:-5],label="rmse weiner filtered") 

        plt.grid(which="both") 
        plt.legend()
        plt.title("Log IPFB RMS residuals (smoothed)\nrmse virgin = {:.3f} rmse weiner = {:.3f}".format(rms_net_virgin,rms_net_weiner),fontsize=16) 
        plt.xlabel("Channel (n)",fontsize=13)
        plt.ylabel("RMSE",fontsize=13)
        plt.tight_layout()
        plt.savefig("Log_virgin_IPFB_RMS_residuals_weiner_{}.png".format(np.random.randint(2)))
        plt.show()
        x_out_3 = conjugate_gradient_descent_verbose(B_3,u_3,x,x0=x0_weiner,rmin=0.0,max_iter=10)
        # above set rmin to zero for it to just iterate specified amount of times
#             input("\nPress [Enter] to proceede\n") 
#         x_out_3 = conjugate_gradient_descent(B_3,u_3,x0=x0_with_zeros,rmin=1000,max_iter=25)
#         print("\n\n-----------------------------------------------------------------------------------------------")
#         print("-----------------------------------------------------------------------------------------------")
#         print("\n\n\nConjugate Gradient Descent, with 1% extra data (prior is a quantized 1% of original timestream)")
    x_out_1 = conjugate_gradient_descent(B_1,u_1,x0=x0_weiner,rmin=2000,max_iter=35)        
    return 